# 실습 4: Agent & Memory 생성

## 목표
- AgentCore Memory 개념 이해
- Memory Manager & Hooks 구현
- Agent Factory 생성

---
## 📚 AgentCore Memory 개념

AgentCore Memory는 AI 에이전트가 **대화 컨텍스트를 기억**하고 **과거 상호작용을 학습**할 수 있게 해주는 서비스입니다.

### Short-term Memory vs Long-term Memory

| 구분 | Short-term Memory | Long-term Memory |
|------|-------------------|------------------|
| **지속성** | 현재 세션 내에서만 유지 | 세션 종료 후에도 영구 저장 |
| **용도** | 현재 대화 흐름 유지 | 사용자 선호도, 과거 이력 저장 |
| **예시** | "아까 말한 주소로..." | "지난번에 문의했던 건..." |
| **저장 방식** | 메모리 내 임시 저장 | Vector DB + Semantic Search |

### Memory Strategy 종류

Long-term Memory는 **Strategy**를 통해 정보를 추출하고 저장합니다:

| Strategy | 설명 | 사용 사례 |
|----------|------|----------|
| **Semantic** | 대화에서 의미 있는 정보를 벡터로 저장 | 유사한 과거 상황 검색 |
| **Summary** | 대화 요약을 자동 생성하여 저장 | 긴 대화 히스토리 압축 |
| **User Profile** | 사용자 선호도/특성 추출 | 개인화된 응답 제공 |

### Namespace 개념

Memory는 **Namespace**로 데이터를 구조화합니다:

```
/actors/{actorId}                    → 사용자별 메모리
/actors/{actorId}/{sessionId}        → 사용자+세션별 메모리  
/summaries/{actorId}/{sessionId}     → 세션 요약 저장
```

### Memory 동작 흐름

```
1. 사용자 메시지 수신
       ↓
2. retrieve_memories() → 관련 과거 기억 검색
       ↓
3. Agent가 컨텍스트와 함께 응답 생성
       ↓
4. create_event() → 현재 대화를 메모리에 저장
       ↓
5. Strategy가 자동으로 Long-term Memory 추출
```

---
## 4-1. 설정 확인

In [ ]:
from workshop_config import REGION, MEMORY_ID, MODEL_ID

print(f"🌍 Region: {REGION}")
print(f"🧠 Memory ID: {MEMORY_ID}")
print(f"🤖 Model: {MODEL_ID}")

if not MEMORY_ID:
    print("\n⚠️ MEMORY_ID가 없습니다. lab2를 먼저 실행하세요.")

## 4-2. 디렉토리 구조 생성

In [ ]:
import os
import shutil

dirs = [
    'agent/deploy/runtime/agent',
    'agent/deploy/runtime/memory',
    'agent/deploy/runtime/tool_use/browser_tool',
]

for d in dirs:
    os.makedirs(d, exist_ok=True)
    init_file = os.path.join(d, '__init__.py')
    if not os.path.exists(init_file):
        open(init_file, 'w').close()

# workshop_config.py를 runtime 폴더에 복사
shutil.copy('workshop_config.py', 'agent/deploy/runtime/')

print("✅ 디렉토리 구조 생성 완료")

---
## 4-3. Memory Manager 생성

Memory Manager는 **세션 관리**를 담당합니다:
- `MemorySessionManager`: Memory 리소스에 대한 세션 관리자
- `MemorySession`: 특정 actor/session에 대한 메모리 세션

In [ ]:
%%writefile agent/deploy/runtime/memory/manager.py
"""Memory session management

MemorySessionManager: Memory 리소스 전체를 관리
MemorySession: 특정 사용자(actor)와 세션에 대한 메모리 접근
"""

from bedrock_agentcore.memory.session import MemorySession, MemorySessionManager
from workshop_config import MEMORY_ID, REGION

_session_manager = None

def get_memory_session_manager() -> MemorySessionManager:
    """싱글톤 패턴으로 세션 매니저 반환"""
    global _session_manager
    if _session_manager is None:
        _session_manager = MemorySessionManager(memory_id=MEMORY_ID, region_name=REGION)
        print(f"✅ Session manager initialized: {MEMORY_ID}")
    return _session_manager

def create_memory_session(actor_id: str, session_id: str) -> MemorySession:
    """특정 사용자/세션에 대한 메모리 세션 생성
    
    Args:
        actor_id: 사용자 식별자 (예: "user-123")
        session_id: 세션 식별자 (예: "session-abc")
    
    Returns:
        MemorySession: 해당 사용자/세션의 메모리에 접근할 수 있는 세션 객체
    """
    manager = get_memory_session_manager()
    session = manager.create_memory_session(actor_id=actor_id, session_id=session_id)
    print(f"🔧 Memory session: actor={actor_id}, session={session_id}")
    return session

---
## 4-4. Memory Hooks 생성

**Hooks**는 Agent 실행 전/후에 자동으로 호출되는 콜백입니다:

| Hook | 시점 | 용도 |
|------|------|------|
| `on_agent_start` | Agent 실행 전 | 과거 메모리 로드, 컨텍스트 주입 |
| `on_agent_end` | Agent 실행 후 | 현재 대화 저장, 메모리 업데이트 |

### Short-term Memory Hook
- 현재 세션의 대화 히스토리 관리
- Agent의 `messages` 배열에 이전 대화 추가

### Long-term Memory Hook  
- `retrieve_memories()`: 관련 과거 기억 검색
- `create_event()`: 현재 대화를 영구 저장

In [ ]:
%%writefile agent/deploy/runtime/memory/hooks.py
"""Memory hooks for agent lifecycle

Hooks는 Agent 실행 전/후에 자동 호출되어 메모리를 관리합니다.
- on_agent_start: 과거 메모리 로드
- on_agent_end: 현재 대화 저장
"""

from strands import HookProvider
from bedrock_agentcore.memory import MemoryClient
from bedrock_agentcore.memory.session import MemorySession

class ShortTermMemoryHookProvider(HookProvider):
    """Short-term Memory Hook
    
    현재 세션 내 대화 컨텍스트를 유지합니다.
    - on_agent_start: 세션 내 이전 대화 로드
    - on_agent_end: 현재 대화를 세션에 추가
    """
    def __init__(self, session: MemorySession):
        self.session = session
    
    def on_agent_start(self, agent, input_data):
        """Agent 실행 전: 세션 컨텍스트 로드
        
        실제 구현 시:
        - self.session에서 이전 메시지 조회
        - agent.messages에 컨텍스트 추가
        """
        pass
    
    def on_agent_end(self, agent, output_data):
        """Agent 실행 후: 현재 대화 저장
        
        실제 구현 시:
        - input_data와 output_data를 세션에 저장
        """
        pass

class LongTermMemoryHookProvider(HookProvider):
    """Long-term Memory Hook
    
    세션을 넘어 영구적인 메모리를 관리합니다.
    - on_agent_start: 관련 과거 기억을 Semantic Search로 검색
    - on_agent_end: 현재 대화를 create_event()로 저장 → Strategy가 자동 추출
    """
    def __init__(self, memory_id: str, memory_client: MemoryClient):
        self.memory_id = memory_id
        self.memory_client = memory_client
    
    def on_agent_start(self, agent, input_data):
        """Agent 실행 전: 관련 Long-term Memory 검색
        
        실제 구현 시:
        memories = self.memory_client.retrieve_memories(
            memory_id=self.memory_id,
            namespace=f"/actors/{actor_id}",
            query=input_data  # 현재 질문과 유사한 과거 기억 검색
        )
        # 검색된 memories를 agent 컨텍스트에 추가
        """
        pass
    
    def on_agent_end(self, agent, output_data):
        """Agent 실행 후: 대화를 Long-term Memory에 저장
        
        실제 구현 시:
        self.memory_client.create_event(
            memory_id=self.memory_id,
            actor_id=actor_id,
            session_id=session_id,
            messages=[
                (user_message, "USER"),
                (assistant_response, "ASSISTANT")
            ]
        )
        # Strategy가 자동으로 의미 있는 정보 추출
        """
        pass

---
## 4-5. Agent Factory 생성

Agent Factory는 **설정된 Agent 인스턴스를 생성**하는 팩토리 패턴입니다:
- Model 설정 (Claude Sonnet)
- System Prompt 설정
- Memory Hooks 연결
- Tools 등록 (lab5에서 추가)

In [ ]:
%%writefile agent/deploy/runtime/agent/factory.py
"""Agent factory for creating configured agents"""

from strands import Agent
from strands.models import BedrockModel
from bedrock_agentcore.memory import MemoryClient

from workshop_config import MODEL_ID, REGION, MEMORY_ID
from memory.manager import create_memory_session
from memory.hooks import ShortTermMemoryHookProvider, LongTermMemoryHookProvider

AGENT_NAME = "FireCommandAssistant"

AGENT_SYSTEM_PROMPT = """당신은 소방서 화재 대응 지휘를 지원하는 전문 AI 어시스턴트입니다.

**핵심 임무:**
1. 화재 현장의 전술적 정보 제공
2. 신속한 의사결정 지원
3. 대응 자원 배치 최적화 제안

**절대 규칙:**
1. 모든 응답은 한글로 작성
2. 도구 사용 시 목적과 결과 명시
3. 소방서 검색 시 <event type="geocode">위도,경도,소방서이름</event> 형식 반환
4. 기상 정보 조회 시 <event type="windy">위도,경도,온도,풍속,풍향,습도,기압,주소</event> 형식 반환
"""

def create_agent(actor_id: str, session_id: str) -> Agent:
    """메모리와 도구가 설정된 에이전트 생성
    
    Args:
        actor_id: 사용자 식별자
        session_id: 세션 식별자
    
    Returns:
        Agent: 설정된 Strands Agent 인스턴스
    """
    # Memory 세션 생성
    user_session = create_memory_session(actor_id, session_id)
    memory_client = MemoryClient(region_name=REGION)
    
    # Bedrock Model 설정
    model = BedrockModel(model_id=MODEL_ID)
    
    # Tools import (lab5에서 생성됨)
    try:
        from tool_use.disaster_tools import search_wikipedia, find_fire_station, get_weather_info
        from strands_tools import browser_tool
        tools = [browser_tool, search_wikipedia, find_fire_station, get_weather_info]
    except ImportError:
        print("⚠️ Tools not found. Run lab5 first.")
        tools = []
    
    # Agent 생성
    agent = Agent(
        name=AGENT_NAME,
        model=model,
        system_prompt=AGENT_SYSTEM_PROMPT,
        hooks=[
            ShortTermMemoryHookProvider(user_session),
            LongTermMemoryHookProvider(MEMORY_ID, memory_client)
        ],
        tools=tools,
        state={"actor_id": actor_id, "session_id": session_id}
    )
    
    print(f"✅ Agent 생성: actor={actor_id}")
    return agent

---
## 📝 Memory 사용 예시 (참고)

실제 Memory API 사용 예시입니다:

```python
from bedrock_agentcore.memory import MemoryClient

client = MemoryClient(region_name="us-west-2")

# 대화 저장
client.create_event(
    memory_id="mem-xxx",
    actor_id="user-123",
    session_id="session-abc",
    messages=[
        ("서초구에서 화재가 발생했습니다", "USER"),
        ("가까운 소방서를 검색하겠습니다...", "ASSISTANT"),
    ]
)

# 관련 기억 검색
memories = client.retrieve_memories(
    memory_id="mem-xxx",
    namespace="/actors/user-123",
    query="이전에 서초구 화재 대응한 적 있나요?"
)
```

---
## ✅ 실습 4 완료!

### 학습한 내용
- Short-term vs Long-term Memory 차이
- Memory Strategy (Semantic, Summary)
- Namespace를 통한 메모리 구조화
- Hooks를 통한 Agent 라이프사이클 관리

### 생성된 파일
- `agent/deploy/runtime/memory/manager.py`
- `agent/deploy/runtime/memory/hooks.py`
- `agent/deploy/runtime/agent/factory.py`

👉 다음: **lab5_create_tools.ipynb**